In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import boto3
import sagemaker
from sagemaker import get_execution_role

In [5]:
import boto3
forecast = boto3.client('forecast')

In [6]:
ds_response = client.create_dataset(
    DatasetName='cvvproj1',
    Domain='RETAIL',
    DatasetType='TARGET_TIME_SERIES',
    DataFrequency='D',
    Schema={
        'Attributes': [
            {
                'AttributeName': 'item_id',
                'AttributeType': 'string'
            },
            {
                'AttributeName': 'timestamp',
                'AttributeType': 'timestamp'
            },
            {
                'AttributeName': 'demand',
                'AttributeType': 'integer'
            },
            {
                'AttributeName': 'branch_code',
                'AttributeType': 'string'
            },
        ]
    }
)

In [7]:
dataset_arn= ds_response['DatasetArn']
print(dataset_arn)

arn:aws:forecast:ap-south-1:989581089309:dataset/cvvproj1


In [8]:
dsgrp_response = client.create_dataset_group(
    DatasetGroupName='cvvprojgrp1',
    Domain='RETAIL',
    DatasetArns=[
        dataset_arn,
    ],
)

In [9]:
print(dsgrp_response)

{'DatasetGroupArn': 'arn:aws:forecast:ap-south-1:989581089309:dataset-group/cvvprojgrp1', 'ResponseMetadata': {'RequestId': 'aa1109dd-4d57-45f3-98ed-7cbcdd574e22', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 14 Nov 2022 13:20:43 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '88', 'connection': 'keep-alive', 'x-amzn-requestid': 'aa1109dd-4d57-45f3-98ed-7cbcdd574e22'}, 'RetryAttempts': 0}}


In [10]:
dsgrp_arn='arn:aws:forecast:ap-south-1:989581089309:dataset-group/cvvprojgrp1'

In [12]:
role_arn="arn:aws:iam::989581089309:role/service-role/AmazonForecast-ExecutionRole-1668330818615"
dsij_response = client.create_dataset_import_job(
    DatasetImportJobName='impccvproj1',
    DatasetArn=dataset_arn,
    DataSource={
        'S3Config': {
            'Path': 's3://ccvdemoproj/consolidated_sample_dataset.csv',
            'RoleArn':role_arn,
        }
    },
    TimestampFormat='yyyy-MM-dd HH:mm:ss',
)

In [29]:
forecastHorizon=24
create_predictor_response=client.create_predictor(PredictorName="ccvprojpredic", 
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML=True,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": dsgrp_arn},
                                                  FeaturizationConfig= {"ForecastFrequency": "D", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "demand", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

In [31]:
predictorArn=create_predictor_response['PredictorArn']
print(predictorArn)

arn:aws:forecast:ap-south-1:989581089309:predictor/ccvprojpredic


In [40]:
predictorArn=create_predictor_response['PredictorArn']
in_progress_predictors = [
     create_predictor_response['PredictorArn']
]
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for predictor_arn in in_progress_predictors:
        predictor_response = client.describe_predictor(
            PredictorArn = predictorArn
        )
        status = predictor_response["Status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(predictorArn))
            in_progress_predictors.remove(predictorArn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(predictorArn))
            in_progress_predictors.remove(predictorArn)
    
    if len(in_progress_predictors) <= 0:
        break
    else:
        print("At least one Predictor build is still in progress")
        
    time.sleep(60)

At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress
At least one Predictor build is still in progress


In [41]:
client.get_accuracy_metrics(PredictorArn=predictorArn)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
   'TestWindows': [{'EvaluationType': 'SUMMARY',
     'Metrics': {'RMSE': 53440.361495732424,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.4495077963957709},
       {'Quantile': 0.5, 'LossValue': 0.534138895485753},
       {'Quantile': 0.1, 'LossValue': 0.18085166597559166}],
      'ErrorMetrics': [{'ForecastType': 'mean',
        'WAPE': 0.526836351030612,
        'RMSE': 53440.361495732424,
        'MASE': 5.169068697659013,
        'MAPE': 10.202728935702023}],
      'AverageWeightedQuantileLoss': 0.38816611928570516}},
    {'TestWindowStart': datetime.datetime(2022, 2, 5, 0, 0, tzinfo=tzlocal()),
     'TestWindowEnd': datetime.datetime(2022, 3, 1, 0, 0, tzinfo=tzlocal()),
     'ItemCount': 15,
     'EvaluationType': 'COMPUTED',
     'Metrics': {'RMSE': 53440.361495732424,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.4495077963957709

In [42]:
forecastName="cvvprojforecast"
create_forecast_response=client.create_forecast(ForecastName=forecastName,
                                                  PredictorArn=predictorArn)
forecastArn = create_forecast_response['ForecastArn']
print(forecastArn)

arn:aws:forecast:ap-south-1:989581089309:forecast/cvvprojforecast


In [43]:
in_progress_forecast = [
    create_forecast_response['ForecastArn']
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for forecast_arn in in_progress_forecast:
        forecast_response = forecast.describe_forecast(
            ForecastArn = forecast_arn
        )
        status = forecast_response["Status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(forecast_arn))
            in_progress_forecast.remove(forecast_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(forecast_arn))
            in_progress_forecast.remove(forecast_arn)
    
    if len(in_progress_forecast) <= 0:
        break
    else:
        print("At least one Forecast build is still in progress")
        
    time.sleep(60)

Build succeeded for arn:aws:forecast:ap-south-1:989581089309:forecast/cvvprojforecast


In [44]:
forecastExportName='Export_forecasts'
outputPath="s3://ccvdemoproj/ccvprojforecastresults"
forecast_export_response = forecast.create_forecast_export_job(
                                                                ForecastExportJobName = forecastExportName,
                                                                ForecastArn=forecastArn, 
                                                                Destination = {
                                                                   "S3Config" : {
                                                                       "Path":outputPath,
                                                                       "RoleArn": role_arn
                                                                   } 
                                                                }
                                                              )

In [47]:
forecastExportJobArn = forecast_export_response['ForecastExportJobArn']
print(forecastExportJobArn)

arn:aws:forecast:ap-south-1:989581089309:forecast-export-job/cvvprojforecast/Export_forecasts
